In [56]:
%pip install composable

Note: you may need to restart the kernel to use updated packages.


In [57]:
from composable import pipeable
from composable.strict import map, filter
import composable_records as rec
import composable_tuples as tup
from composable_glob import glob
from composable_utility import get, with_open, identity, apply
from composable_object import obj, attr
from composable_origami import fold
from operator import add

import re
import csv

# Project Overview
## - Merging the Attendance files with composable functions - 

### Goals for part 3:
    1. Combine all attendance files into one CSV file.
    2. Add the course and section information as columns in the resulting file. 
    3. Use a single pipe with composable functions and records to accomplish this part. 
      (The writing of the output file can be in a separate cell). 
    4. You will need to make your own headers based on the README iles.
    5. 
There is no missing data, so you can use string split to split up the fixed widt
    6.         
        -   Feel free to split the date-time column into two separate columns when splitting, just make sure you adjust the header acrdin-- l
        - .
BONUS: Do this using unfold instead of string 
    6. See the following videos for (A) and overview of the composable records verbs and (B) for a demonstration of a similar problem.      - 
Activity 2.7.3 - Understanding composable rec.
       - - 
Activity 2.7.3 - Combining pitching yearly files with composable functionssplit.

## Import csv files

In [59]:
(paths :=
 "./attendance_example_fixed_width/*/attendance*.csv"
 >> glob(recursive=True)
)

['./attendance_example_fixed_width/dsci494s7/attendance_quiz.csv',
 './attendance_example_fixed_width/stat180s18/attendance_quiz.csv',
 './attendance_example_fixed_width/stat491s1/attendance_quiz.csv']

In [66]:
#? map

In [114]:
get_out_path = lambda r: r['folder'] + '_Part3_' + r['type_'] + '.csv'

(attendance_records :=
 paths
 >> map(rec.create (path = identity,                           # use map - outside is a list[]
                    lines =  with_open(obj.readlines()),       # read in lines, you could use lambda
                   )
       ) 
 >> map(rec.update(len_lines = lambda r: len(r.lines),         # get the length of lines 
                  )
       )
 >> filter(lambda r: r.len_lines > 1)   
 >> map(rec.update(file = lambda r: r.path >> obj.split('/') >> get(-1),
                   #folder = lambda r: r.path.split('/attendance_quiz') >> get(0) >> apply(lambda s: s + '/'),
                   folder = lambda r: re.split(r'/[A-Za-z]{4}\d{3}', r.path) >> get(0) >> apply(lambda s: s + '/'),
                   temp_class = lambda r: r.path.split('/') >> get(2), 
                   class_ = lambda r: re.split(r'\d{3}', r.temp_class) >> get(0),
                   section = lambda r: r.temp_class.split('s') >> get(-1) >> apply(lambda s: 's' + s),
                   type_ = lambda r: r.file.split('_') >> get(0),
                   header = lambda r: r.lines >> tup.first,
                   body = lambda r: r.lines >> tup.rest,
                   sequential = True # What is the pupose of this code?
                  )
       )
 >> map(rec.apply(header = lambda s: 'Class,Section,' + s))
 >> map(rec.update(body = lambda r: r.body >> map(lambda line: str(r.class_) + ',' + 
                                                               str(r.section) + ',' + line),
                  )
       )
 >> rec.zip_at(['type_', 'folder', 'header', 'body']) #Zip up the relevant keys.
 >> rec.apply(
              folder = tup.first, # Extract the folder, type, and header
              type_ = tup.first,
              header = tup.first,
             )
 >> rec.update(body = lambda r: r >> get('body') >> fold(add, [r['header']]) #Fold the bodies starting with the header
             ) 
 >> rec.update(out_path = get_out_path) # How can I take this out?
) 

{'type_': 'attendance',
 'folder': './attendance_example_fixed_width/',
 'header': 'Class,Section,ID,UserName,FirstName,LastName,Attempt,Score,Out_Of,Attempt_Start_Date,Attempt_Start_Time,Attempt_End_Date,Attempt_End_Time,Percent\n',
 'body': ['Class,Section,ID,UserName,FirstName,LastName,Attempt,Score,Out_Of,Attempt_Start_Date,Attempt_Start_Time,Attempt_End_Date,Attempt_End_Time,Percent\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,1,1,1,2019-01-14,14:00:00,2019-01-14,14:06:00,100%\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,2,1,1,2019-01-16,14:00:00,2019-01-16,14:08:00,100%\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,3,1,1,2019-01-18,14:00:00,2019-01-18,14:05:00,100%\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,4,1,1,2019-01-23,14:00:00,2019-01-23,14:06:00,100%\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,5,1,1,2019-01-25,14:00:00,2019-01-25,14:10:00,100%\n',
  'dsci,s7,14460432,au9747cp,Jericho,Greer,6,1,1,2019-01-28,14:04:00,2019-01-28,14:12:00,100%\n',
  'dsci,s7,144604

## Saving as a single csv

In [125]:
# list of dictionaly won't work like a list?
#output_file = attendance_records[0] 
#output_file = attendance_records.values(0)
output_file = next(iter(attendance_records.values())) # dictionary won't accept index like a list
# output_file = (iter(attendance_records.values())) >> get(0)
output_file

'attendance'

In [119]:
output_file = attendance_records['out_path']

with open(output_file, mode = 'w') as out:
    out.writelines(attendance_records['body'])
    print(f"CSV file saved: {output_file}")

CSV file saved: ./attendance_example_fixed_width/_Part3_attendance.csv
